# Learning with Massive Data
<p>
Assignment 3 - Similarity search for document pairs<br>
Giovanni Costa - 880892
</p>

<p>
<b>SEQUENTIAL VERSION</b>
</p>

Contents:
- [Document sparse representation](#doc_repr)
- [Sequential Implementation](#s_impl)
    - [Exact similarity search](#exact_s)
    - [Approximate similarity search](#approx_s)
- [Evaluations](#eval)

In [9]:
import pandas as pd
import numpy as np
from utils import compute_sparse_repr, eval_sol
from scipy.sparse import load_npz, save_npz
from matplotlib import pyplot as plt
from sklearn.random_projection import SparseRandomProjection

<a id="doc_repr"></a>
## Document sparse representation

In [10]:
results="results/"
datasets=["datasets/nfcorpus/corpus.jsonl", "datasets/scifact/corpus.jsonl"]

In [11]:
df_docs1=pd.read_json(datasets[0], lines=True)
df_docs2=pd.read_json(datasets[1], lines=True)

In [12]:
df_docs1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3633 entries, 0 to 3632
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   _id       3633 non-null   object
 1   title     3633 non-null   object
 2   text      3633 non-null   object
 3   metadata  3633 non-null   object
dtypes: object(4)
memory usage: 113.7+ KB


In [13]:
df_docs2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5183 entries, 0 to 5182
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   _id       5183 non-null   int64 
 1   title     5183 non-null   object
 2   text      5183 non-null   object
 3   metadata  5183 non-null   object
dtypes: int64(1), object(3)
memory usage: 162.1+ KB


In [14]:
sparse_repr1, vocab1, idf1=compute_sparse_repr(df_docs1["text"])
sparse_repr2, vocab2, idf2=compute_sparse_repr(df_docs2["text"])

In [15]:
sparse_repr1.shape

(3633, 18867)

In [16]:
#pd.DataFrame(vocab_1, columns=["terms"]).to_parquet("terms_nfcorpus.parquet")
pd.DataFrame(df_docs1["_id"]).to_parquet(results+"ids_nfcorpus.parquet")
pd.DataFrame(((1+df_docs1.shape[0])/(10**(idf1-1)))-1, columns=["df_t"]).to_parquet(results+"doc_freq_nfcorpus.parquet")
save_npz(results+"sparse_repr_nfcorpus.npz", sparse_repr1)

pd.DataFrame(df_docs2["_id"]).to_parquet(results+"ids_scifact.parquet")
pd.DataFrame(((1+df_docs2.shape[0])/(10**(idf2-1)))-1, columns=["df_t"]).to_parquet(results+"doc_freq_scifact.parquet")
save_npz(results+"sparse_repr_scifact.npz", sparse_repr2)

<a id="s_impl"></a>
## Sequential Implementation
<a id="exact_s"></a>
### Exact similarity search

In [9]:
sparse_repr1=load_npz(results+"sparse_repr_nfcorpus.npz")
print(sparse_repr1.shape)
print("Density ratio:", sparse_repr1.count_nonzero()/(sparse_repr1.shape[0]*sparse_repr1.shape[1]))

(3633, 18867)
Density ratio: 0.006536315875404127


In [10]:
sparse_repr2=load_npz(results+"sparse_repr_scifact.npz")
print(sparse_repr2.shape)
print("Density ratio:", sparse_repr2.count_nonzero()/(sparse_repr2.shape[0]*sparse_repr2.shape[1]))

(5183, 26109)
Density ratio: 0.004343720063974072


In [11]:
thresholds=[0.3, 0.5, 0.8]

In [12]:
time_list1, pairs_list1=eval_sol(sparse_repr1, df_docs1["_id"], thresholds)

Threshold: 0.3
N. of pairs: 13026
N. of pairs: 13026
N. of pairs: 13026
Mean time elapsed: 5.07207179069519

Threshold: 0.5
N. of pairs: 832
N. of pairs: 832
N. of pairs: 832
Mean time elapsed: 4.316487471262614

Threshold: 0.8
N. of pairs: 44
N. of pairs: 44
N. of pairs: 44
Mean time elapsed: 4.6432632605234785



In [13]:
time_list2, pairs_list2=eval_sol(sparse_repr2, df_docs2["_id"], thresholds)

Threshold: 0.3
N. of pairs: 9524
N. of pairs: 9524
N. of pairs: 9524
Mean time elapsed: 8.82019329071045

Threshold: 0.5
N. of pairs: 729
N. of pairs: 729
N. of pairs: 729
Mean time elapsed: 9.055848280588785

Threshold: 0.8
N. of pairs: 7
N. of pairs: 7
N. of pairs: 7
Mean time elapsed: 8.524721066157023



<a id="approx_s"></a>
### Approximate similarity search
(using Sparse Random Projection)

In [14]:
epsilon=0.1

In [15]:
sr_proj1=SparseRandomProjection(eps=epsilon, random_state=32)
sr_proj1.fit(sparse_repr1);
print(sr_proj1.n_components_)
print(sr_proj1.density_)

7026
0.0072802882585279016


In [16]:
sparse_repr_approx_srp1=sr_proj1.transform(sparse_repr1)
print(sparse_repr_approx_srp1.shape)
print("Density ratio:", sparse_repr_approx_srp1.count_nonzero()/(sparse_repr_approx_srp1.shape[0]*sparse_repr_approx_srp1.shape[1]))

(3633, 7026)
Density ratio: 0.5847442580658102


In [17]:
sr_proj2=SparseRandomProjection(eps=epsilon, random_state=32)
sr_proj2.fit(sparse_repr2);
print(sr_proj2.n_components_)
print(sr_proj2.density_)

7331
0.006188777667238989


In [18]:
sparse_repr_approx_srp2=sr_proj2.transform(sparse_repr2)
print(sparse_repr_approx_srp2.shape)
print("Density ratio:", sparse_repr_approx_srp2.count_nonzero()/(sparse_repr_approx_srp2.shape[0]*sparse_repr_approx_srp2.shape[1]))

(5183, 7331)
Density ratio: 0.49419999008857984


In [19]:
time_list1_appprox, pairs_list1_approx=eval_sol(sparse_repr_approx_srp1, df_docs1["_id"], thresholds)

Threshold: 0.3
N. of pairs: 13782
N. of pairs: 13782
N. of pairs: 13782
Mean time elapsed: 138.38636994361877

Threshold: 0.5
N. of pairs: 963
N. of pairs: 963
N. of pairs: 963
Mean time elapsed: 97.02835456530254

Threshold: 0.8
N. of pairs: 45
N. of pairs: 45
N. of pairs: 45
Mean time elapsed: 100.21871733665466



In [20]:
time_list2_appprox, pairs_list2_approx=eval_sol(sparse_repr_approx_srp2, df_docs2["_id"], thresholds)

Threshold: 0.3
N. of pairs: 9560
N. of pairs: 9560
N. of pairs: 9560
Mean time elapsed: 161.7169984181722

Threshold: 0.5
N. of pairs: 841
N. of pairs: 841
N. of pairs: 841
Mean time elapsed: 161.16109538078308

Threshold: 0.8
N. of pairs: 6
N. of pairs: 6
N. of pairs: 6
Mean time elapsed: 158.42339706420898



<a id="eval"></a>
## Results

In [41]:
for i in range(len(thresholds)):
    print(f"Threshold: {thresholds[i]}")
    exact_set1=set([(e[0], e[1]) for e in pairs_list1[i]])
    approx_set1=set([(e[0], e[1]) for e in pairs_list1_approx[i]])
    jaccard=len(exact_set1.intersection(approx_set1))/len(exact_set1.union(approx_set1))
    print(f"Jaccard score: {jaccard}")
    print(f"Time for exact solution: {time_list1[i]}")
    print(f"Time for approx solution: {time_list1_appprox[i]}\n")


Threshold: 0.3
Jaccard score: 0.7215515026971487
Time for exact solution: [5.0446531772613525, 5.602407932281494, 4.569154262542725]
Time for approx solution: [139.88466215133667, 138.89840292930603, 136.37604475021362]

Threshold: 0.5
Jaccard score: 0.6437728937728938
Time for exact solution: [4.264678716659546, 4.290329217910767, 4.394454479217529]
Time for approx solution: [102.54280090332031, 93.05330562591553, 95.48895716667175]

Threshold: 0.8
Jaccard score: 0.8936170212765957
Time for exact solution: [4.264760255813599, 4.735339641571045, 4.929689884185791]
Time for approx solution: [99.54363107681274, 102.52391338348389, 98.58860754966736]



In [42]:
for i in range(len(thresholds)):
    print(f"Threshold: {thresholds[i]}")
    exact_set2=set([(e[0], e[1]) for e in pairs_list2[i]])
    approx_set2=set([(e[0], e[1]) for e in pairs_list2_approx[i]])
    jaccard=len(exact_set2.intersection(approx_set2))/len(exact_set2.union(approx_set2))
    print(f"Jaccard score: {jaccard}")
    print(f"Time for exact solution: {time_list2[i]}")
    print(f"Time for approx solution: {time_list2_appprox[i]}\n")


Threshold: 0.3
Jaccard score: 0.7533994854832782
Time for exact solution: [8.810563802719116, 9.178160905838013, 8.471855163574219]
Time for approx solution: [156.66780543327332, 165.60336446762085, 162.87982535362244]

Threshold: 0.5
Jaccard score: 0.5874620829120324
Time for exact solution: [8.992414712905884, 9.024078130722046, 9.151051998138428]
Time for approx solution: [170.2303969860077, 156.69716024398804, 156.55572891235352]

Threshold: 0.8
Jaccard score: 0.625
Time for exact solution: [8.86352014541626, 8.545639038085938, 8.165004014968872]
Time for approx solution: [153.4403212070465, 158.14087867736816, 163.68899130821228]

